In [1]:
import pandas as pd
import numpy as np

# Motivation
The Goal is to create vector data bases that gets filled with data about the football players which allows to determine similar players based on their attributes and characteristics. This notebook shall help to build the basic functionality to achieve this goal. These are the steps which are covered in this notebook.
* Build a Vector Database
* Load Players into the Database
* Perform testing queries
* Draw conclusion

The first step is to load the preprocessed data.

In [2]:
df = pd.read_csv('../data/preprocessed_data.csv', sep=',')
display(df)

,Unnamed: 0,ID,Overall,Wage,International Reputation,Weak Foot,Skill Moves,Height,Weight,Crossing,...,Position_RDM,Position_RES,Position_RF,Position_RM,Position_RS,Position_RW,Position_RWB,Position_ST,Position_SUB,Position_nan
0,0,212198,3.197778,12.429220,3.0,3.0,4.0,-0.353843,-0.869188,1.977395,...,0,0,0,0,0,0,0,0,0,0
1,1,209658,3.041584,11.849405,4.0,4.0,3.0,1.098401,0.949142,1.299699,...,0,0,0,0,0,0,0,0,0,0
2,2,176580,3.197778,11.813037,5.0,4.0,3.0,0.081830,1.089013,1.582072,...,0,0,0,0,1,0,0,0,0,0
3,3,192985,3.666359,12.765691,4.0,5.0,4.0,-0.063395,-0.729316,2.372717,...,0,0,0,0,0,0,0,0,0,0
4,4,224334,2.573002,10.714440,2.0,3.0,4.0,-1.370415,-0.869188,1.977395,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15813,16703,259718,-2.425202,6.478510,1.0,3.0,1.0,0.953177,1.089013,-2.201730,...,0,1,0,0,0,0,0,0,0,0
15814,16704,251433,-1.488038,6.857514,1.0,2.0,1.0,1.969748,1.368756,-2.484103,...,0,1,0,0,0,0,0,0,0,0
15815,16706,262846,-2.269008,6.311735,1.0,2.0,1.0,1.824523,1.508628,-2.201730,...,0,1,0,0,0,0,0,0,0,0
15816,16707,241317,-3.206171,6.552508,1.0,2.0,1.0,-0.208619,-0.589445,-2.258204,...,0,1,0,0,0,0,0,0,0,0
